In [3]:
import tensorflow as tf

# Sprawdzenie dostępności GPU
physical_devices = tf.config.list_physical_devices('GPU')
print("Dostępne urządzenia GPU:", physical_devices)

# Sprawdzenie wersji TensorFlow
print("Wersja TensorFlow:", tf.__version__)

# Jeśli GPU nie jest dostępne, wypisujemy szczegóły
if not physical_devices:
    print("Brak dostępnych urządzeń GPU. Sprawdź, czy masz poprawnie zainstalowane CUDA i cuDNN.")
else:
    # Jeśli GPU jest dostępne, próbujemy ustawić pamięć dynamiczną
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Pamięć dla GPU ustawiona na dynamiczną.")

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Source - https://stackoverflow.com/a
# Posted by adelriosantiago, modified by community. See post 'Timeline' for change history
# Retrieved 2025-11-26, License - CC BY-SA 4.0

from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

# Your output is probably something like ['/device:CPU:0']
# It should be ['/device:CPU:0', '/device:GPU:0']



TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [5]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import seaborn as sns


TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
import os
import shutil
import random
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img

SOURCE_DIR = r"C:\Users\matik\Documents\Data Sets\Faulty_solar_panel"
TARGET_DIR = r"C:\Users\matik\Documents\Data Sets\Faulty_solar_panel_balanced"

IMAGES_PER_CLASS = 250
IMG_SIZE = (224, 224)

# Używamy bardziej zaawansowanej augmentacji
datagen = ImageDataGenerator(
    rotation_range=45,                  # Losowy obrót w zakresie 45°
    width_shift_range=0.2,              # Losowe przesunięcie w poziomie (do 20% szerokości)
    height_shift_range=0.2,             # Losowe przesunięcie w pionie (do 20% wysokości)
    shear_range=0.2,                    # Zniekształcenie obrazu
    zoom_range=0.2,                     # Zmiana zoomu (powiększenie)
    horizontal_flip=True,               # Losowe odbicie w poziomie
    vertical_flip=True,                 # Losowe odbicie w pionie
    brightness_range=[0.8, 1.2],        # Zmiana jasności obrazu
    fill_mode='nearest'                 # Sposób uzupełniania brakujących pikseli
)

# Tworzymy katalog docelowy, jeśli jeszcze nie istnieje
os.makedirs(TARGET_DIR, exist_ok=True)

for class_name in os.listdir(SOURCE_DIR):
    src_class_path = os.path.join(SOURCE_DIR, class_name)
    tgt_class_path = os.path.join(TARGET_DIR, class_name)

    if not os.path.isdir(src_class_path):
        continue

    os.makedirs(tgt_class_path, exist_ok=True)

    images = os.listdir(src_class_path)
    random.shuffle(images)

    # 1. Kopiowanie oryginalnych obrazów do folderu docelowego (max 300)
    base_images = images[:min(len(images), IMAGES_PER_CLASS)]

    count = 0
    for img_name in base_images:
        shutil.copy(
            os.path.join(src_class_path, img_name),
            os.path.join(tgt_class_path, img_name)
        )
        count += 1

    # 2. Generowanie nowych obrazów przez augmentację, jeśli brakuje do 300
    while count < IMAGES_PER_CLASS:
        img_name = random.choice(base_images)
        img_path = os.path.join(src_class_path, img_name)

        # Ładowanie obrazu
        img = load_img(img_path, target_size=IMG_SIZE)
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)

        # Generowanie nowych obrazów przez augmentację
        aug_iter = datagen.flow(x, batch_size=1)

        # Zapis augmentowanego obrazu
        aug_img = next(aug_iter)[0]
        save_img(
            os.path.join(tgt_class_path, f"aug_{count}.jpg"),
            aug_img
        )
        count += 1

    print(f"{class_name}: {count} images")


In [ ]:
TARGET_DIR = r"C:\Users\matik\Documents\Data Sets\Faulty_solar_panel_balanced"

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    TARGET_DIR,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

valid_gen = datagen.flow_from_directory(
    TARGET_DIR,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
# Ścieżka do głównego folderu z danymi
base_dir = r"C:\Users\matik\Documents\Data Sets\Faulty_solar_panel"

# Przygotowanie generatorów danych (treningowych i walidacyjnych)
# datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    base_dir, 
    target_size=(224, 224),  # Rozmiar wejściowy dla ResNet50
    batch_size=32,
    class_mode='categorical',  # Klasyfikacja wieloklasowa
    subset='training'  # Używamy 80% danych do treningu
)

valid_gen = datagen.flow_from_directory(
    base_dir, 
    target_size=(224, 224),  # Rozmiar wejściowy dla ResNet50
    batch_size=32,
    class_mode='categorical',  # Klasyfikacja wieloklasowa
    subset='validation'  # Używamy 20% danych do walidacji
)


In [ ]:
# Ładowanie wstępnie wytrenowanego modelu ResNet50 bez głowy (bo dodamy swoje warstwy)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Zamrażanie warstw bazowych (żeby nie trenować ich ponownie)
base_model.trainable = False

# # Budowanie modelu

# v1
model = models.Sequential([
    base_model,  # Wstępnie wytrenowany model ResNet50
    layers.GlobalAveragePooling2D(),  # Agregacja cech w wektorze
    layers.Dense(512, activation='relu'),  # Warstwa gęsta
    layers.Dense(train_gen.num_classes, activation='softmax')  # Wyjście z klasyfikacją
])

# v2
# model = models.Sequential([
#     base_model,  # Wstępnie wytrenowany model ResNet50
#     layers.GlobalAveragePooling2D(),  # Agregacja cech w wektorze
#     layers.Dense(512, activation='relu'),  # Warstwa gęsta
#     #layers.Dropout(0.2),  # Regularizacja: losowe wyłączanie neuronów
#     layers.Dense(256, activation='relu'),  # Druga warstwa gęsta
#     #layers.Dropout(0.2),  # Kolejny Dropout
#     layers.Dense(train_gen.num_classes, activation='softmax')  # Wyjście z klasyfikacją
# ])

# v3  do bani model
# model = models.Sequential([
#     base_model,
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(256, activation='relu'),
#     layers.BatchNormalization(),
#     layers.Dropout(0.4),
#     layers.Dense(train_gen.num_classes, activation='softmax')
# ])



# Kompilacja modelu
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Podsumowanie modelu
model.summary()


In [ ]:
# Trenowanie modelu
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // train_gen.batch_size,
    epochs=100,
    validation_data=valid_gen,
    validation_steps=valid_gen.samples // valid_gen.batch_size,
    #callbacks=[early_stopping]
)


In [ ]:
# Ocena modelu
loss, accuracy = model.evaluate(valid_gen)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


In [ ]:
# Rysowanie wykresów dla straty
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Rysowanie wykresów dla dokładności
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


In [ ]:
# Pobieranie prawdziwych etykiet
true_labels = valid_gen.classes

# Pobieranie przewidywań modelu
pred_probs = model.predict(valid_gen)
pred_labels = np.argmax(pred_probs, axis=1)

# Obliczanie confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Wypisanie raportu
print("Classification Report:")
print(classification_report(true_labels, pred_labels, target_names=list(train_gen.class_indices.keys())))

# Rysowanie confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d",
            xticklabels=train_gen.class_indices.keys(),
            yticklabels=train_gen.class_indices.keys())
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
model.save('v2_65.h5')